***
*Project:* Helmholtz Machine on Niche Construction

*Author:* Jingwei Liu, Computer Music Ph.D., UC San Diego

*Supervisor:* Shlomo Dubnov, Professor in Music and CSE department, UC San Diego
***

# <span style="background-color:darkorange; color:white; padding:2px 6px">Document 5</span> 

# Single Sample Experiment

*Updated:* May 24, 2023


In [2]:
import numpy as np

### 1. Ground Truth

In [5]:
n = 10
preferred_set = np.zeros([1,10])
preferred_set[0,:2] = 1
preferred_set[0,-2:] = 1

for i in range(2, n-2):
    for j in range(np.shape(preferred_set)[0]):
        
        prefix = preferred_set[j,i-2:i]
        if np.array_equal(prefix, [0,0]) or np.array_equal(prefix, [0,1]):
            preferred_set[j,i] = 1
        else:
            preferred_set = np.append(preferred_set, preferred_set[j:j+1,:], axis=0)
            preferred_set[j,i] = 1
preferred_set = (preferred_set - 0.5)*2
preferred_set

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.],
       [ 1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.],
       [ 1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.],
       [ 1.,  1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.],
       [ 1.,  1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.],
       [ 1.,  1., -1.,  1.,  1., -1., -1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.],
       [ 1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1.

In [6]:
np.shape(preferred_set)

(25, 10)

In [91]:
n = 10
well_formed_set = np.zeros([1,10])
well_formed_set[0,0] = 1

for i in range(1,n):
    for j in range(np.shape(well_formed_set)[0]):
        if i == 2 and np.array_equal(well_formed_set[j,i-2:i], [1,0]):
            well_formed_set[j,i] = 1
        elif i > 3 and np.array_equal(well_formed_set[j,i-3:i], [0,0,0]):
            well_formed_set[j,i] = 1
        elif i > 3 and np.array_equal(well_formed_set[j,i-4:i], [0,0,1,0]):
            well_formed_set[j,i] = 1
        else:
            well_formed_set = np.append(well_formed_set, well_formed_set[j:j+1,:], axis=0)
            well_formed_set[j,i] = 1
            
ind = np.array([], dtype=np.int8)
for i in range(well_formed_set.shape[0]):
    if np.array_equal(well_formed_set[i,-3:], [0,0,1]):
        ind = np.append(ind,i)

well_formed_set = np.delete(well_formed_set,ind,0)
well_formed_set = (well_formed_set - 0.5)*2
well_formed_set

array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1., -1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1., -1., ...,  1.,  1.,  1.],
       ...,
       [ 1.,  1.,  1., ..., -1., -1., -1.],
       [ 1., -1.,  1., ..., -1., -1., -1.],
       [ 1.,  1., -1., ..., -1., -1., -1.]])

There are $2^{10} = 1024$ possible sequences in total, thus the Venn diagram for the ground truth is 

<img src="Venn_1.jpg" style="width:550px;height:500px;">
<caption><center> **Figure 2**: Venn Diagram for Ground Truth Data  </center></caption>

### 2. Experiments

The Helmholtz machine is designed as follows

<img src="niche.jpg">
<caption><center> **Figure 1**: Helmholtz Machine in Niche Construction  </center></caption>

where

- Input layer $d_i$ with 10 neurons $i = 1, \dots, 10, 2^{10} = 1024$ possibilities
- Hidden layer $x_l$ wiht 8 neurons $l = 1, \dots, 8, 2^{8} = 256$ possibilities
- Cause layer $y_j$ wiht 5 neurons $j = 1, \dots, 5, 2^{5} = 32$ possibilities
- Hyper layer $z_k$ wiht 3 neurons $k = 1, \dots, 3, 2^{3} = 8$ possibilities
- Generative bias is always $1$ (**Ursatz** in Schenkerian analysis)

#### 2.2 Functions

In [9]:
def sigmoid(x):
    y = 1/(1+np.exp(-x))
    return y

In [10]:
def wake_forward(data,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,n_x,n_y,n_z):
    q_2 = sigmoid(np.matmul(Phi_12,data) + b_12)
    x = ((q_2 > np.random.rand(n_x,1)).astype(int) - 0.5)*2    # rejection sampling
    # x = np.where(q_2 > 0.5,1,-1)  # deterministic
    
    q_3 = sigmoid(np.matmul(Phi_23,x) + b_23)
    y = ((q_3 > np.random.rand(n_y,1)).astype(int) - 0.5)*2
    
    q_4 = sigmoid(np.matmul(Phi_34,y) + b_34)
    z = ((q_4 > np.random.rand(n_z,1)).astype(int) - 0.5)*2
    
    Q_2 = (np.cumprod(q_2[np.where(x == 1)])[-1] if q_2[np.where(x == 1)].size != 0 else 1) * (np.cumprod(1-q_2[np.where(x == -1)])[-1] if q_2[np.where(x == -1)].size != 0 else 1)*(1.5**n_x)
    Q_3 = (np.cumprod(q_3[np.where(y == 1)])[-1] if q_3[np.where(y == 1)].size != 0 else 1) * (np.cumprod(1-q_3[np.where(y == -1)])[-1] if q_3[np.where(y == -1)].size != 0 else 1)*(1.5**n_y)
    Q_4 = (np.cumprod(q_4[np.where(z == 1)])[-1] if q_4[np.where(z == 1)].size != 0 else 1) * (np.cumprod(1-q_4[np.where(z == -1)])[-1] if q_4[np.where(z == -1)].size != 0 else 1)*(1.5**n_z)
    Q = Q_2 * Q_3 * Q_4
    
    return q_2,q_3,q_4,x,y,z,Q

In [11]:
def sleep_forward(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d):
    p_4 = sigmoid(Theta)
    z = ((p_4 > np.random.rand(n_z,1)).astype(int) - 0.5)*2    # rejection sampling
    
    p_3 = sigmoid(np.matmul(Theta_43,z) + b_43)
    y = ((p_3 > np.random.rand(n_y,1)).astype(int) - 0.5)*2
    
    p_2 = sigmoid(np.matmul(Theta_32,y) + b_32)
    x = ((p_2 > np.random.rand(n_x,1)).astype(int) - 0.5)*2
    
    p_1 = sigmoid(np.matmul(Theta_21,x) + b_21)
    d = ((p_1 > np.random.rand(n_d,1)).astype(int) - 0.5)*2  # sampling
    # d = np.where(p_1 > 0.5,1,-1)  # deterministic
    
    P_4 = (np.cumprod(p_4[np.where(z == 1)])[-1] if p_4[np.where(z == 1)].size != 0 else 1)* (np.cumprod(1-p_4[np.where(z == -1)])[-1] if p_4[np.where(z == -1)].size != 0 else 1)*(1.5**n_z)
    P_3 = (np.cumprod(p_3[np.where(y == 1)])[-1] if p_3[np.where(y == 1)].size != 0 else 1)* (np.cumprod(1-p_3[np.where(y == -1)])[-1] if p_3[np.where(y == -1)].size != 0 else 1)*(1.5**n_y)
    P_2 = (np.cumprod(p_2[np.where(x == 1)])[-1] if p_2[np.where(x == 1)].size != 0 else 1)* (np.cumprod(1-p_2[np.where(x == -1)])[-1] if p_2[np.where(x == -1)].size != 0 else 1)*(1.5**n_x)
    P_1 = (np.cumprod(p_1[np.where(d == 1)])[-1] if p_1[np.where(d == 1)].size != 0 else 1)* (np.cumprod(1-p_1[np.where(d == -1)])[-1] if p_1[np.where(d == -1)].size != 0 else 1)*(1.5**n_d)
    
    P = P_1 * P_2 * P_3 * P_4
    
    return p_4,p_3,p_2,p_1,z,y,x,d,P

In [12]:
def wake_update_delta(z,y,x,d,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr):
    p_4 = sigmoid(Theta)
    Theta -= lr * (p_4 - (1+z)/2)
    
    p_3 = sigmoid(np.matmul(Theta_43,z) + b_43)
    Theta_43 -= lr * np.outer((p_3 - (1+y)/2), z)
    b_43 -= lr * (p_3 - (1+y)/2)
    
    p_2 = sigmoid(np.matmul(Theta_32,y) + b_32)
    Theta_32 -= lr * np.outer((p_2 - (1+x)/2), y)
    b_32 -= lr * (p_2 - (1+x)/2)
    
    p_1 = sigmoid(np.matmul(Theta_21,x) + b_21)
    Theta_21 -= lr * np.outer((p_1 - (1+d)/2), x)
    b_21 -= lr * (p_1 - (1+d)/2)
    
    return Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21

In [13]:
def sleep_update_delta(z,y,x,d,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr):
    
    q_2 = sigmoid(np.matmul(Phi_12,d) + b_12)
    Phi_12 -= lr * np.outer((q_2 - (1+x)/2), d)
    b_12 -= lr * (q_2 - (1+x)/2)
    
    q_3 = sigmoid(np.matmul(Phi_23,x) + b_23)
    Phi_23 -= lr * np.outer((q_3 - (1+y)/2), x)
    b_23 -= lr * (q_3 - (1+y)/2)
    
    q_4 = sigmoid(np.matmul(Phi_34,y) + b_34)
    Phi_34 -= lr * np.outer((q_4 - (1+z)/2), y)
    b_34 -= lr * (q_4 - (1+z)/2)
    
    return Phi_12,Phi_23,Phi_34,b_12,b_23,b_34

In [14]:
def wake_update_delta_weighted(z,y,x,d,Q,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr):
    
    p_4 = sigmoid(Theta)
    Theta -= lr * Q * (p_4 - (1+z)/2)
    
    p_3 = sigmoid(np.matmul(Theta_43,z) + b_43)
    Theta_43 -= lr * Q * np.outer((p_3 - (1+y)/2), z)
    b_43 -= lr * Q * (p_3 - (1+y)/2)
    
    p_2 = sigmoid(np.matmul(Theta_32,y) + b_32)
    Theta_32 -= lr * Q * np.outer((p_2 - (1+x)/2), y)
    b_32 -= lr * Q * (p_2 - (1+x)/2)
    
    p_1 = sigmoid(np.matmul(Theta_21,x) + b_21)
    Theta_21 -= lr * Q * np.outer((p_1 - (1+d)/2), x)
    b_21 -= lr * Q * (p_1 - (1+d)/2)
    
    return Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21

In [15]:
def sleep_update_delta_weighted(z,y,x,d,P,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr):
    
    q_2 = sigmoid(np.matmul(Phi_12,d) + b_12)
    Phi_12 -= lr * P * np.outer((q_2 - (1+x)/2), d)
    b_12 -= lr * P * (q_2 - (1+x)/2)
    
    q_3 = sigmoid(np.matmul(Phi_23,x) + b_23)
    Phi_23 -= lr * P * np.outer((q_3 - (1+y)/2), x)
    b_23 -= lr * P * (q_3 - (1+y)/2)
    
    q_4 = sigmoid(np.matmul(Phi_34,y) + b_34)
    Phi_34 -= lr * P * np.outer((q_4 - (1+z)/2), y)
    b_34 -= lr * P * (q_4 - (1+z)/2)
    
    return Phi_12,Phi_23,Phi_34,b_12,b_23,b_34

In [16]:
def sleep_forward_batch(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d,batch_size):
    p_4 = sigmoid(Theta)
    z = ((p_4 > np.random.rand(n_z,batch_size)).astype(int) - 0.5)*2    # rejection sampling
    
    p_3 = sigmoid(np.matmul(Theta_43,z) + b_43)
    y = ((p_3 > np.random.rand(n_y,batch_size)).astype(int) - 0.5)*2
    
    p_2 = sigmoid(np.matmul(Theta_32,y) + b_32)
    x = ((p_2 > np.random.rand(n_x,batch_size)).astype(int) - 0.5)*2
    
    p_1 = sigmoid(np.matmul(Theta_21,x) + b_21)
    d = ((p_1 > np.random.rand(n_d,batch_size)).astype(int) - 0.5)*2
    
    P_4 = np.cumprod(np.where(z == 1,p_4,1),axis=0)[-1] * np.cumprod(np.where(z == -1,1-p_4,1),axis=0)[-1]*(1.5**n_z)
    P_3 = np.cumprod(np.where(y == 1,p_3,1),axis=0)[-1] * np.cumprod(np.where(y == -1,1-p_3,1),axis=0)[-1]*(1.5**n_y)
    P_2 = np.cumprod(np.where(x == 1,p_2,1),axis=0)[-1] * np.cumprod(np.where(x == -1,1-p_2,1),axis=0)[-1]*(1.5**n_x)
    P_1 = np.cumprod(np.where(d == 1,p_1,1),axis=0)[-1] * np.cumprod(np.where(d == -1,1-p_1,1),axis=0)[-1]*(1.5**n_d)
    
    P = P_1 * P_2 * P_3 * P_4
    
    return p_4,p_3,p_2,p_1,z,y,x,d,P

#### 2.1 Parameter Initialization

In [17]:
n_z = 3
n_y = 5
n_x = 8
n_d = 10

In [18]:
# Random initialization

Phi_12 = np.random.rand(n_x,n_d) *2-1
Phi_23 = np.random.rand(n_y,n_x) *2-1
Phi_34 = np.random.rand(n_z,n_y) *2-1
b_12 = np.random.rand(n_x,1) *2-1
b_23 = np.random.rand(n_y,1) *2-1
b_34 = np.random.rand(n_z,1) *2-1

Theta = np.random.rand(n_z,1) *2-1
Theta_43 = np.random.rand(n_y,n_z) *2-1
Theta_32 = np.random.rand(n_x,n_y) *2-1
Theta_21 = np.random.rand(n_d,n_x) *2-1
b_43 = np.random.rand(n_y,1) *2-1
b_32 = np.random.rand(n_x,1) *2-1
b_21 = np.random.rand(n_d,1) *2-1

In [19]:
# Zero initialization

Phi_12 = np.zeros((n_x,n_d))
Phi_23 = np.zeros((n_y,n_x))
Phi_34 = np.zeros((n_z,n_y))
b_12 = np.zeros((n_x,1))
b_23 = np.zeros((n_y,1))
b_34 = np.zeros((n_z,1))

Theta = np.zeros((n_z,1))
Theta_43 = np.zeros((n_y,n_z))
Theta_32 = np.zeros((n_x,n_y))
Theta_21 = np.zeros((n_d,n_x))
b_43 = np.zeros((n_y,1))
b_32 = np.zeros((n_x,1))
b_21 = np.zeros((n_d,1))

In [20]:
print ("Phi_12: " + str(Phi_12))
print ("Phi_23: " + str(Phi_23))
print ("Phi_34: " + str(Phi_34))
print ("b_12: " + str(b_12))
print ("b_23: " + str(b_23))
print ("b_34: " + str(b_34))

Phi_12: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Phi_23: [[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
Phi_34: [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
b_12: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
b_23: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
b_34: [[0.]
 [0.]
 [0.]]


In [21]:
print ("Theta: " + str(Theta))
print ("Theta_43: " + str(Theta_43))
print ("Theta_32: " + str(Theta_32))
print ("Theta_21: " + str(Theta_21))
print ("b_43: " + str(b_43))
print ("b_32: " + str(b_32))
print ("b_21: " + str(b_21))

Theta: [[0.]
 [0.]
 [0.]]
Theta_43: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Theta_32: [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Theta_21: [[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
b_43: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
b_32: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
b_21: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


#### 2.3 Training

In [22]:
train_set = np.transpose(well_formed_set)
train_set.shape

(10, 256)

In [23]:
lr = 0.01
epoch = 10

In [24]:
Phi_12_cache = np.copy(Phi_12)
Phi_23_cache = np.copy(Phi_23)
Phi_34_cache = np.copy(Phi_34)
b_12_cache = np.copy(b_12)
b_23_cache = np.copy(b_23)
b_34_cache = np.copy(b_34)

Theta_cache = np.copy(Theta)
Theta_43_cache = np.copy(Theta_43)
Theta_32_cache = np.copy(Theta_32)
Theta_21_cache = np.copy(Theta_21)
b_43_cache = np.copy(b_43)
b_32_cache = np.copy(b_32)
b_21_cache = np.copy(b_21)

In [25]:
# Sample-based training
# Local delta rule
for e in range(epoch):
    num_samples = train_set.shape[1]
    train_set = train_set[:,np.random.permutation(num_samples)]
    
    for i in range(num_samples):
        data = train_set[:,i:i+1]
        q_2,q_3,q_4,x_w,y_w,z_w,Q = wake_forward(data,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,n_x,n_y,n_z)
        Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21 = wake_update_delta(z_w,y_w,x_w,data,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr)
        p_4,p_3,p_2,p_1,z_s,y_s,x_s,d_s,P = sleep_forward(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d)
        Phi_12,Phi_23,Phi_34,b_12,b_23,b_34 = sleep_update_delta(z_s,y_s,x_s,d_s,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr)

    recog_phi = np.sum((Phi_12 - Phi_12_cache)**2) + np.sum((Phi_23 - Phi_23_cache)**2) + np.sum((Phi_34 - Phi_34_cache)**2) 
    recog_b = np.sum((b_12 - b_12_cache)**2) + np.sum((b_23 - b_23_cache)**2) + np.sum((b_34 - b_34_cache)**2)
    gener_theta = np.sum((Theta - Theta_cache)**2) + np.sum((Theta_43 - Theta_43_cache)**2) + np.sum((Theta_32 - Theta_32_cache)**2) + np.sum((Theta_21 - Theta_21_cache)**2)
    gener_b = np.sum((b_43 - b_43_cache)**2) + np.sum((b_32 - b_32_cache)**2) + np.sum((b_21 - b_21_cache)**2)
    print ("recog_phi: " + str(recog_phi))
    print ("recog_b: " + str(recog_b))
    print ("gener_theta: " + str(gener_theta))
    print ("gener_b: " + str(gener_b))
    print('')
    
    Phi_12_cache = np.copy(Phi_12)
    Phi_23_cache = np.copy(Phi_23)
    Phi_34_cache = np.copy(Phi_34)
    b_12_cache = np.copy(b_12)
    b_23_cache = np.copy(b_23)
    b_34_cache = np.copy(b_34)

    Theta_cache = np.copy(Theta)
    Theta_43_cache = np.copy(Theta_43)
    Theta_32_cache = np.copy(Theta_32)
    Theta_21_cache = np.copy(Theta_21)
    b_43_cache = np.copy(b_43)
    b_32_cache = np.copy(b_32)
    b_21_cache = np.copy(b_21)
    

recog_phi: 0.5278343205890828
recog_b: 0.06773745205921824
gener_theta: 0.5247527994250258
gener_b: 1.4000741714444334

recog_phi: 0.6844551973574243
recog_b: 0.052660068459397816
gener_theta: 0.6088347026421053
gener_b: 1.1674213602286703

recog_phi: 0.545773577229776
recog_b: 0.10543820484727884
gener_theta: 0.6906909182174604
gener_b: 1.1416271877225102

recog_phi: 0.6607595799573603
recog_b: 0.0840353478290182
gener_theta: 0.7659866011941036
gener_b: 0.7438823196312874

recog_phi: 0.6194136692823695
recog_b: 0.06766370050302825
gener_theta: 0.6944877183118023
gener_b: 1.5852356328649413

recog_phi: 0.5437435516814214
recog_b: 0.054862567432751336
gener_theta: 0.729105451603815
gener_b: 1.0014820923424366

recog_phi: 0.5685328999934512
recog_b: 0.059690211527544876
gener_theta: 0.7625081012158575
gener_b: 0.6421799408087538

recog_phi: 0.759839849864565
recog_b: 0.14264987053037154
gener_theta: 0.7262494647788666
gener_b: 0.4020322241661389

recog_phi: 0.7553151342086318
recog_b: 0.

In [26]:
print ("Phi_12: " + str(Phi_12))
print ("Phi_12_cache: " + str(Phi_12_cache))
print ("Phi_23: " + str(Phi_23))
print ("Phi_34: " + str(Phi_34))
print ("b_12: " + str(b_12))
print ("b_12_cache: " + str(b_12_cache))
print ("b_23: " + str(b_23))
print ("b_34: " + str(b_34))

Phi_12: [[ 0.01269094  0.08083073 -0.33741222  0.22574226  0.22196432  0.02084409
   0.09840083 -0.19056728 -0.07662406 -0.18836081]
 [ 0.11409806 -0.2900551   0.04338411  0.03775277 -0.03048923 -0.22424151
  -0.2109864   0.1903914   0.0173626  -0.14135155]
 [-0.26998221 -0.00530961 -0.14752935 -0.08597504 -0.09100886  0.05028176
  -0.0944872  -0.2053698  -0.04158364  0.10034402]
 [ 0.28203606 -0.02737827  0.05730215 -0.07281887 -0.06462181  0.05099204
  -0.02054051  0.18146126  0.15031234 -0.05797329]
 [ 0.02497399  0.08854256 -0.00866404  0.02017437 -0.06149543 -0.09807922
  -0.06304517  0.27244518 -0.16097253  0.14635416]
 [-0.21195625 -0.17571827 -0.10649344  0.11263758 -0.27275239  0.37912878
   0.3220075   0.20435035 -0.03906301  0.08978343]
 [ 0.06552464  0.18665177  0.0709286   0.04118831  0.06897558  0.24974118
  -0.12773212  0.1899817  -0.11489227 -0.08983448]
 [-0.02301479  0.10170016 -0.10646368 -0.23993506  0.17242982 -0.26678978
   0.01427172  0.09989383  0.01885934  0.22

In [27]:
print ("Theta: " + str(Theta))
print ("Theta_43: " + str(Theta_43))
print ("Theta_32: " + str(Theta_32))
print ("Theta_21: " + str(Theta_21))
print ("b_43: " + str(b_43))
print ("b_32: " + str(b_32))
print ("b_21: " + str(b_21))

Theta: [[-0.12217145]
 [ 0.24169577]
 [ 0.19546966]]
Theta_43: [[ 0.10024706  0.27534641  0.21101696]
 [-0.12589586 -0.03081535 -0.126246  ]
 [-0.06335384  0.1273391   0.0527027 ]
 [ 0.23820324 -0.02301156  0.40356071]
 [ 0.14773894  0.12859999  0.12961827]]
Theta_32: [[-0.02104694  0.09708293 -0.00505736  0.36729245  0.2599389 ]
 [ 0.06952781  0.02063866  0.02925366 -0.19896809  0.1020676 ]
 [-0.12821052 -0.00872108 -0.00931587  0.04265344 -0.20767271]
 [-0.31890651  0.12299659  0.11043773 -0.11313152 -0.04460242]
 [-0.08963751 -0.09559329  0.10858764  0.01825911 -0.06670121]
 [-0.15777583  0.18736284 -0.17633823  0.00821422 -0.19340131]
 [ 0.31011713  0.02495789  0.21986423  0.04483086 -0.09155359]
 [ 0.01673651 -0.14768291  0.01477214  0.08268317  0.03630028]]
Theta_21: [[ 0.0248763   0.0922205  -0.19252678  0.26304035 -0.19901236 -0.20207646
   0.00155397 -0.06545104]
 [ 0.27419298 -0.31451541 -0.01870987 -0.13220658  0.11061607 -0.21794075
   0.14526793 -0.01211787]
 [-0.3101829  

#### 2.4 Testing

In [128]:
test_size = 100

In [129]:
def generation(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d,batch_size):
    p_4 = sigmoid(Theta)
    z = ((p_4 > np.random.rand(n_z,batch_size)).astype(int) - 0.5)*2    # rejection sampling
    
    p_3 = sigmoid(np.matmul(Theta_43,z) + b_43)
    y = ((p_3 > np.random.rand(n_y,batch_size)).astype(int) - 0.5)*2
    
    p_2 = sigmoid(np.matmul(Theta_32,y) + b_32)
    x = ((p_2 > np.random.rand(n_x,batch_size)).astype(int) - 0.5)*2
    # x = np.where(p_2 > 0.5,1,-1)
    
    p_1 = sigmoid(np.matmul(Theta_21,x) + b_21)
    # d = ((p_1 > np.random.rand(n_d,batch_size)).astype(int) - 0.5)*2
    d = np.where(p_1 > 0.5,1,-1)
    
    P_4 = np.cumprod(np.where(z == 1,p_4,1),axis=0)[-1] * np.cumprod(np.where(z == -1,1-p_4,1),axis=0)[-1]*(1.5**n_z)
    P_3 = np.cumprod(np.where(y == 1,p_3,1),axis=0)[-1] * np.cumprod(np.where(y == -1,1-p_3,1),axis=0)[-1]*(1.5**n_y)
    P_2 = np.cumprod(np.where(x == 1,p_2,1),axis=0)[-1] * np.cumprod(np.where(x == -1,1-p_2,1),axis=0)[-1]*(1.5**n_x)
    P_1 = np.cumprod(np.where(d == 1,p_1,1),axis=0)[-1] * np.cumprod(np.where(d == -1,1-p_1,1),axis=0)[-1]*(1.5**n_d)
    
    P = P_1 * P_2 * P_3 * P_4
    
    return p_4,p_3,p_2,p_1,z,y,x,d,P

In [130]:
p_4_gen,p_3_gen,p_2_gen,p_1_gen,z_s_gen,y_s_gen,x_s_gen,d_s_gen,P_gen = generation(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d,test_size)
print ("p_4_gen: " + str(p_4_gen[:,0:10]))
print ("p_3_gen: " + str(p_3_gen[:,0:10]))
print ("p_2_gen: " + str(p_2_gen[:,0:10]))
print ("p_1_gen: " + str(p_1_gen[:,0:10]))
print ("z_s_gen: " + str(z_s_gen[:,0:10]))
print ("y_s_gen: " + str(y_s_gen[:,0:10]))
print ("x_s_gen: " + str(x_s_gen[:,0:10]))
print ("d_s_gen: " + str(d_s_gen[:,0:10]))
print ("P_gen: " + str(P_gen[0:10]))

p_4_gen: [[0.3294903 ]
 [0.27233043]
 [0.44289062]]
p_3_gen: [[0.30817678 0.24037749 0.30817678 0.24178852 0.30817678 0.1027548
  0.30817678 0.24178852 0.55174302 0.1027548 ]
 [0.74072593 0.34366951 0.74072593 0.4731572  0.74072593 0.67480047
  0.74072593 0.4731572  0.41891791 0.67480047]
 [0.73860602 0.6853437  0.73860602 0.56622165 0.73860602 0.80019904
  0.73860602 0.56622165 0.60578689 0.80019904]
 [0.32358227 0.57375755 0.32358227 0.45747034 0.32358227 0.19238977
  0.32358227 0.45747034 0.72995561 0.19238977]
 [0.45284208 0.46698078 0.45284208 0.60251117 0.45284208 0.61016369
  0.45284208 0.60251117 0.31659513 0.61016369]]
p_2_gen: [[0.17241702 0.57957026 0.17153076 0.26099509 0.17241702 0.04445418
  0.47642158 0.04505287 0.47642158 0.17153076]
 [0.05208408 0.38452675 0.4354822  0.34576433 0.05208408 0.24044855
  0.09554491 0.29984136 0.09554491 0.4354822 ]
 [0.48464898 0.6785059  0.49379921 0.36612868 0.48464898 0.38913097
  0.49648007 0.39441562 0.49648007 0.49379921]
 [0.431081

#### Check with the rules (well-formedness rules)

1. Start with 1
2. Forbid 00100 (no 100, 001 on the boundary)
3. Forbid 0000


In [131]:
test_set = (d_s_gen+1)/2
test_set[:,0:10]

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 0., 1., 1., 0., 1., 1., 0.],
       [1., 0., 1., 1., 0., 1., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 0., 0., 0., 1.],
       [0., 1., 0., 1., 1., 0., 1., 1., 0., 0.],
       [1., 1., 1., 0., 1., 1., 0., 1., 0., 1.],
       [1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 1., 1., 0., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 0., 0., 1.],
       [0., 0., 1., 1., 0., 0., 1., 1., 0., 1.]])

In [132]:
st, counts = np.unique(test_set, axis=1,return_counts=True)
print(st)
print(counts)
st.shape

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
  1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1.]
 [0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1.
  1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
  1. 1. 1.]
 [1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1.
  1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1.
  1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1.

(10, 51)

In [133]:
violation_rule1 = 0
violation_rule1_index = []
violation_rule2 = 0
violation_rule2_index = []
violation_rule2_bd = 0
violation_rule2_bd_index = []
violation_rule3 = 0
violation_rule3_index = []
valid_num = 0
valid_index = []

pattern_len = test_set.shape[0]
for i in range(test_set.shape[1]):
    flag = 0
    pattern = test_set[:,i]
    if pattern[0] != 1:
        violation_rule1 += 1
        violation_rule1_index.append(i)
        flag = 1
    if np.array_equal(pattern[0:3], [1,0,0]) or np.array_equal(pattern[-3:], [0,0,1]):
        violation_rule2_bd += 1
        violation_rule2_bd_index.append(i)
        flag = 1
    for j in range(pattern_len - 5):
        if np.array_equal(pattern[j:j+5],[0,0,1,0,0]):
            violation_rule2 += 1
            violation_rule2_index.append(i)
            flag = 1
            break
    for j in range(pattern_len - 4):
        if np.array_equal(pattern[j:j+4],[0,0,0,0]):
            violation_rule3 += 1
            violation_rule3_index.append(i)
            flag = 1
            break
    if flag == 0:
        valid_num += 1
        valid_index.append(i)

In [134]:
print ("violation_rule1: " + str(violation_rule1))
print ("violation_rule1_index: " + str(violation_rule1_index))
print ("violation_rule2: " + str(violation_rule2))
print ("violation_rule2_index: " + str(violation_rule2_index))
print ("violation_rule2_bd: " + str(violation_rule2_bd))
print ("violation_rule2_bd_index: " + str(violation_rule2_bd_index))
print ("violation_rule3: " + str(violation_rule3))
print ("violation_rule3_index: " + str(violation_rule3_index))
print ("valid_num: " + str(valid_num))
print ("valid_index: " + str(valid_index))
print ("valid_percentage: " + str(valid_num/test_size))

violation_rule1: 0
violation_rule1_index: []
violation_rule2: 0
violation_rule2_index: []
violation_rule2_bd: 0
violation_rule2_bd_index: []
violation_rule3: 0
violation_rule3_index: []
valid_num: 100
valid_index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
valid_percentage: 1.0


In [135]:
test_set[:,6]

array([1., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

#### Train with selective sampling

We fine-tune the pre-trained model with valid generations within the well-formed set, shrinking the input data by active sampling to construct a niche within the well-formed set.

In [45]:
preferred_set.shape

(25, 10)

In [94]:
init_set = np.transpose(well_formed_set)
init_set.shape

(10, 256)

In [95]:
well_formed_set

array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1., -1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1., -1., ...,  1.,  1.,  1.],
       ...,
       [ 1.,  1.,  1., ..., -1., -1., -1.],
       [ 1., -1.,  1., ..., -1., -1., -1.],
       [ 1.,  1., -1., ..., -1., -1., -1.]])

In [48]:
def wake_forward(data,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,n_x,n_y,n_z):
    q_2 = sigmoid(np.matmul(Phi_12,data) + b_12)
    x = ((q_2 > np.random.rand(n_x,1)).astype(int) - 0.5)*2    # rejection sampling
    # x = np.where(q_2 > 0.5,1,-1)  # deterministic
    
    q_3 = sigmoid(np.matmul(Phi_23,x) + b_23)
    y = ((q_3 > np.random.rand(n_y,1)).astype(int) - 0.5)*2
    
    q_4 = sigmoid(np.matmul(Phi_34,y) + b_34)
    z = ((q_4 > np.random.rand(n_z,1)).astype(int) - 0.5)*2
    
    Q_2 = (np.cumprod(q_2[np.where(x == 1)])[-1] if q_2[np.where(x == 1)].size != 0 else 1) * (np.cumprod(1-q_2[np.where(x == -1)])[-1] if q_2[np.where(x == -1)].size != 0 else 1)*(1.5**n_x)
    Q_3 = (np.cumprod(q_3[np.where(y == 1)])[-1] if q_3[np.where(y == 1)].size != 0 else 1) * (np.cumprod(1-q_3[np.where(y == -1)])[-1] if q_3[np.where(y == -1)].size != 0 else 1)*(1.5**n_y)
    Q_4 = (np.cumprod(q_4[np.where(z == 1)])[-1] if q_4[np.where(z == 1)].size != 0 else 1) * (np.cumprod(1-q_4[np.where(z == -1)])[-1] if q_4[np.where(z == -1)].size != 0 else 1)*(1.5**n_z)
    Q = Q_2 * Q_3 * Q_4
    
    return q_2,q_3,q_4,x,y,z,Q

In [49]:
def sleep_forward(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d):
    p_4 = sigmoid(Theta)
    z = ((p_4 > np.random.rand(n_z,1)).astype(int) - 0.5)*2    # rejection sampling
    
    p_3 = sigmoid(np.matmul(Theta_43,z) + b_43)
    y = ((p_3 > np.random.rand(n_y,1)).astype(int) - 0.5)*2
    
    p_2 = sigmoid(np.matmul(Theta_32,y) + b_32)
    x = ((p_2 > np.random.rand(n_x,1)).astype(int) - 0.5)*2
    
    p_1 = sigmoid(np.matmul(Theta_21,x) + b_21)
    d = ((p_1 > np.random.rand(n_d,1)).astype(int) - 0.5)*2  # sampling
    # d = np.where(p_1 > 0.5,1,-1)  # deterministic
    
    P_4 = (np.cumprod(p_4[np.where(z == 1)])[-1] if p_4[np.where(z == 1)].size != 0 else 1)* (np.cumprod(1-p_4[np.where(z == -1)])[-1] if p_4[np.where(z == -1)].size != 0 else 1)*(1.5**n_z)
    P_3 = (np.cumprod(p_3[np.where(y == 1)])[-1] if p_3[np.where(y == 1)].size != 0 else 1)* (np.cumprod(1-p_3[np.where(y == -1)])[-1] if p_3[np.where(y == -1)].size != 0 else 1)*(1.5**n_y)
    P_2 = (np.cumprod(p_2[np.where(x == 1)])[-1] if p_2[np.where(x == 1)].size != 0 else 1)* (np.cumprod(1-p_2[np.where(x == -1)])[-1] if p_2[np.where(x == -1)].size != 0 else 1)*(1.5**n_x)
    P_1 = (np.cumprod(p_1[np.where(d == 1)])[-1] if p_1[np.where(d == 1)].size != 0 else 1)* (np.cumprod(1-p_1[np.where(d == -1)])[-1] if p_1[np.where(d == -1)].size != 0 else 1)*(1.5**n_d)
    
    P = P_1 * P_2 * P_3 * P_4
    
    return p_4,p_3,p_2,p_1,z,y,x,d,P

In [50]:
lr = 0.01
epoch = 10
gen_size = 10

In [51]:
Phi_12_cache = np.copy(Phi_12)
Phi_23_cache = np.copy(Phi_23)
Phi_34_cache = np.copy(Phi_34)
b_12_cache = np.copy(b_12)
b_23_cache = np.copy(b_23)
b_34_cache = np.copy(b_34)

Theta_cache = np.copy(Theta)
Theta_43_cache = np.copy(Theta_43)
Theta_32_cache = np.copy(Theta_32)
Theta_21_cache = np.copy(Theta_21)
b_43_cache = np.copy(b_43)
b_32_cache = np.copy(b_32)
b_21_cache = np.copy(b_21)

In [52]:
 # check if "d_s_gen" is in the well-formed set
def check(d_s_gen):
    d_s = 0
    pattern_len = d_s_gen.shape[0]
    for j in range(d_s_gen.shape[1]):
        flag = 0
        pattern = d_s_gen[:,j]
        if pattern[0] != 1:
            flag = 1
            continue
        if np.array_equal(pattern[0:3], [1,-1,-1]) or np.array_equal(pattern[-3:], [-1,-1,1]):
            flag = 1
            continue
        for k in range(pattern_len - 5):
            if np.array_equal(pattern[k:k+5],[-1,-1,1,-1,-1]):
                flag = 1
                break
        if flag == 1:
            continue
        for k in range(pattern_len - 4):
            if np.array_equal(pattern[k:k+4],[-1,-1,-1,-1]):
                flag = 1
                break

        if flag == 0:
            d_s = d_s_gen[:,j:j+1]
            break
    return d_s,j,flag

In [370]:
## Sample-based training
## Local delta rule
#
#for e in range(epoch):
#    num_samples = train_set.shape[1]
#    train_set = train_set[:,np.random.permutation(num_samples)]
#    for i in range(num_samples):
#        data = init_set[:,i:i+1]
#        q_2,q_3,q_4,x_w,y_w,z_w,Q = wake_forward(data,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,n_x,n_y,n_z)
#        Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21 = wake_update_delta(z_w,y_w,x_w,data,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr)
#        
#        # check if "d_s_gen" is in the well-formed set
#        flag = 1
#        while flag == 1:
#            p_4_gen,p_3_gen,p_2_gen,p_1_gen,z_s_gen,y_s_gen,x_s_gen,d_s_gen,P_gen = sleep_forward_batch(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d,gen_size)
#            d_s,j,flag = check(d_s_gen)
#        
#        
#        Phi_12,Phi_23,Phi_34,b_12,b_23,b_34 = sleep_update_delta(z_s_gen[:,j:j+1],y_s_gen[:,j:j+1],x_s_gen[:,j:j+1],d_s,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr)
#
#        
#    recog_phi = np.sum((Phi_12 - Phi_12_cache)**2) + np.sum((Phi_23 - Phi_23_cache)**2) + np.sum((Phi_34 - Phi_34_cache)**2) 
#    recog_b = np.sum((b_12 - b_12_cache)**2) + np.sum((b_23 - b_23_cache)**2) + np.sum((b_34 - b_34_cache)**2)
#    gener_theta = np.sum((Theta - Theta_cache)**2) + np.sum((Theta_43 - Theta_43_cache)**2) + np.sum((Theta_32 - Theta_32_cache)**2) + np.sum((Theta_21 - Theta_21_cache)**2)
#    gener_b = np.sum((b_43 - b_43_cache)**2) + np.sum((b_32 - b_32_cache)**2) + np.sum((b_21 - b_21_cache)**2)
#    
#    print ("recog_phi: " + str(recog_phi))
#    print ("recog_b: " + str(recog_b))
#    print ("gener_theta: " + str(gener_theta))
#    print ("gener_b: " + str(gener_b))
#    print('')
#    
#    Phi_12_cache = np.copy(Phi_12)
#    Phi_23_cache = np.copy(Phi_23)
#    Phi_34_cache = np.copy(Phi_34)
#    b_12_cache = np.copy(b_12)
#    b_23_cache = np.copy(b_23)
#    b_34_cache = np.copy(b_34)
#
#    Theta_cache = np.copy(Theta)
#    Theta_43_cache = np.copy(Theta_43)
#    Theta_32_cache = np.copy(Theta_32)
#    Theta_21_cache = np.copy(Theta_21)
#    b_43_cache = np.copy(b_43)
#    b_32_cache = np.copy(b_32)
#    b_21_cache = np.copy(b_21)
#    


recog_phi: 0.7349899693855896
recog_b: 0.0813074131317329
gener_theta: 0.6462999096540598
gener_b: 0.5125110833739853

recog_phi: 0.5987970005071059
recog_b: 0.05086063070849918
gener_theta: 0.7014794593430442
gener_b: 0.36441255894450203

recog_phi: 0.6670757257884118
recog_b: 0.047587619836476434
gener_theta: 0.8474320839651933
gener_b: 1.6334136648855495

recog_phi: 0.4642463466402632
recog_b: 0.06245242370118425
gener_theta: 0.6120000876487652
gener_b: 0.5164247299010561

recog_phi: 0.5617871581795073
recog_b: 0.04229802735489639
gener_theta: 0.6013339969229351
gener_b: 1.2905311484224713

recog_phi: 0.6157211074597526
recog_b: 0.06682830630299874
gener_theta: 0.8019779129267365
gener_b: 1.3737961931059228

recog_phi: 0.46947315883304896
recog_b: 0.0559122999588843
gener_theta: 0.6997650442020075
gener_b: 1.0162307320850723

recog_phi: 0.6975212483934968
recog_b: 0.07255134167401417
gener_theta: 0.6991822214406869
gener_b: 0.8074166868114242

recog_phi: 0.5506299959697056
recog_b: 

In [43]:
itr = 1000

In [119]:
# Sample-based training
# Local delta rule
# Sample environment


for i in range(itr):
    sample = np.random.randint(init_set.shape[1])
    data = init_set[:,sample:sample+1]
    q_2,q_3,q_4,x_w,y_w,z_w,Q = wake_forward(data,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,n_x,n_y,n_z)
    Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21 = wake_update_delta(z_w,y_w,x_w,data,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr)

    # check if "d_s_gen" is in the well-formed set
    flag = 1
    while flag == 1:
        p_4_gen,p_3_gen,p_2_gen,p_1_gen,z_s_gen,y_s_gen,x_s_gen,d_s_gen,P_gen = sleep_forward_batch(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d,gen_size)
        d_s,j,flag = check(d_s_gen)


    Phi_12,Phi_23,Phi_34,b_12,b_23,b_34 = sleep_update_delta(z_s_gen[:,j:j+1],y_s_gen[:,j:j+1],x_s_gen[:,j:j+1],d_s,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34,lr)
    init_set = np.append(init_set,d_s,axis=1)

    
    if i%100 == 0:
        
        recog_phi = np.sum((Phi_12 - Phi_12_cache)**2) + np.sum((Phi_23 - Phi_23_cache)**2) + np.sum((Phi_34 - Phi_34_cache)**2) 
        recog_b = np.sum((b_12 - b_12_cache)**2) + np.sum((b_23 - b_23_cache)**2) + np.sum((b_34 - b_34_cache)**2)
        gener_theta = np.sum((Theta - Theta_cache)**2) + np.sum((Theta_43 - Theta_43_cache)**2) + np.sum((Theta_32 - Theta_32_cache)**2) + np.sum((Theta_21 - Theta_21_cache)**2)
        gener_b = np.sum((b_43 - b_43_cache)**2) + np.sum((b_32 - b_32_cache)**2) + np.sum((b_21 - b_21_cache)**2)

        print ("recog_phi: " + str(recog_phi))
        print ("recog_b: " + str(recog_b))
        print ("gener_theta: " + str(gener_theta))
        print ("gener_b: " + str(gener_b))
        print('')

        Phi_12_cache = np.copy(Phi_12)
        Phi_23_cache = np.copy(Phi_23)
        Phi_34_cache = np.copy(Phi_34)
        b_12_cache = np.copy(b_12)
        b_23_cache = np.copy(b_23)
        b_34_cache = np.copy(b_34)

        Theta_cache = np.copy(Theta)
        Theta_43_cache = np.copy(Theta_43)
        Theta_32_cache = np.copy(Theta_32)
        Theta_21_cache = np.copy(Theta_21)
        b_43_cache = np.copy(b_43)
        b_32_cache = np.copy(b_32)
        b_21_cache = np.copy(b_21)


recog_phi: 0.19630824462182533
recog_b: 0.033274577180828026
gener_theta: 0.2174204709789268
gener_b: 0.04694824737319091

recog_phi: 0.1974373711534326
recog_b: 0.022394477787204648
gener_theta: 0.23034748004321065
gener_b: 0.03228536976830171

recog_phi: 0.2074176906199024
recog_b: 0.021132176326729733
gener_theta: 0.2451829527040998
gener_b: 0.05962253755930787

recog_phi: 0.14407807856124097
recog_b: 0.01617998424295098
gener_theta: 0.20753410264652386
gener_b: 0.02747368226665267

recog_phi: 0.18974339475355934
recog_b: 0.02913015550984792
gener_theta: 0.25157954772801805
gener_b: 0.04854065349110813

recog_phi: 0.20565373565515277
recog_b: 0.020447132776876153
gener_theta: 0.24203902140701067
gener_b: 0.02249866377929991

recog_phi: 0.1949093479902472
recog_b: 0.03865683647852858
gener_theta: 0.2636950784822019
gener_b: 0.04408732802328894

recog_phi: 0.28333814307876204
recog_b: 0.027981415211649367
gener_theta: 0.20568617669739636
gener_b: 0.03847589867210321

recog_phi: 0.1787

In [120]:
init_set.shape

(10, 2256)

In [121]:
np.unique(init_set,axis = 1).shape

(10, 271)

In [124]:
init_set[0,:]

array([1., 1., 1., ..., 1., 1., 1.])

In [125]:
d_s

array([[ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.]])

In [126]:
values, counts = np.unique(init_set, axis = 1, return_counts=True)

In [127]:
counts

array([ 5,  7,  6,  9,  5,  8,  2,  1,  8, 12,  1,  9, 12,  4, 16,  9, 17,
        1,  2,  4,  9,  4,  6,  4,  9,  5, 10, 15,  7,  9,  3,  2,  4, 11,
        8,  9,  1,  5,  7, 10, 18, 14, 18,  5,  5,  3,  8,  9,  6,  6, 10,
       14, 11,  1,  6,  8,  4,  7,  8,  8,  4,  8, 10, 12, 12, 23, 18,  3,
        3,  2,  8,  8,  8,  3,  4,  3,  7, 12,  6, 10,  2,  4,  3,  6, 11,
        8,  7,  3,  5,  9, 10, 12, 14, 13,  4,  2,  3,  4,  3,  4,  3, 13,
        6,  9,  3,  6,  4,  3, 13,  8, 15,  7,  2,  5,  2, 10,  8,  4,  3,
        3,  4,  3, 16,  9, 12, 10,  4,  4,  6,  2,  4,  1,  7,  3,  4,  6,
        2,  2,  3,  4,  3,  2,  4,  3,  2,  6,  7,  9,  6,  2,  2,  5,  2,
        8,  4,  1,  3,  4,  9,  9, 13,  4,  4,  3,  2,  8,  4,  9,  7,  6,
        9,  5, 11, 11, 10,  9, 10,  3, 14, 12,  8,  8,  3,  9,  5, 12,  5,
       10,  6, 15, 11,  8, 24,  1,  5,  4, 11,  9, 11,  5,  7,  9, 16, 15,
       16, 22,  2,  4,  4, 17, 18,  9, 12,  9,  6,  6, 14, 32, 20, 23,  8,
        4,  7, 11,  5,  8

In [103]:
j

0

In [104]:
flag

0

In [105]:
d_s_gen

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1.,  1.],
       [ 1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.],
       [-1.,  1., -1.,  1.,  1., -1.,  1., -1.,  1., -1.],
       [ 1.,  1.,  1., -1., -1.,  1., -1., -1., -1.,  1.],
       [ 1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.],
       [ 1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.],
       [ 1., -1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1.],
       [ 1., -1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.]])

In [381]:
print ("Phi_12: " + str(Phi_12))
print ("Phi_12_cache: " + str(Phi_12_cache))
print ("Phi_23: " + str(Phi_23))
print ("Phi_34: " + str(Phi_34))
print ("b_12: " + str(b_12))
print ("b_12_cache: " + str(b_12_cache))
print ("b_23: " + str(b_23))
print ("b_34: " + str(b_34))

Phi_12: [[ 6.70304076e-02 -3.73388615e-01 -1.63960751e-01 -2.16131417e-01
  -9.97556859e-03  2.76467603e-01  4.19593708e-01 -2.78304046e-02
   1.30092957e-02 -1.51694461e-01]
 [-1.41964575e-01  7.99138412e-02  7.94223034e-02 -1.50949491e-01
  -1.15934233e-01 -1.20204384e-01 -5.44691138e-02  1.68590382e-01
  -5.69710922e-02 -6.88087712e-02]
 [-2.87411222e-02  2.30729124e-01  1.82844179e-01 -1.85021868e-01
  -5.16913403e-02 -5.90269264e-02  6.32247240e-02  1.30343294e-01
  -2.53688233e-01 -1.05411366e-01]
 [-8.60785949e-02  6.58448419e-02  6.94285080e-02 -7.02602679e-02
  -2.75067542e-01  8.98170289e-02 -3.07255035e-01  8.23371208e-02
  -1.41944336e-01 -8.70622018e-03]
 [ 2.96104232e-01 -5.58831697e-02 -4.57095438e-02  1.25584588e-01
  -1.52799171e-01 -2.24888644e-01  2.08746679e-01  1.97274501e-01
   2.36990478e-01 -1.63690976e-02]
 [ 8.73696020e-03  3.25784046e-01 -6.95272142e-02 -1.33765298e-01
   8.27284299e-03 -2.41048524e-01  3.18509507e-01  3.34778127e-01
  -1.86048194e-02 -1.6325

In [382]:
print ("Theta: " + str(Theta))
print ("Theta_43: " + str(Theta_43))
print ("Theta_32: " + str(Theta_32))
print ("Theta_21: " + str(Theta_21))
print ("b_43: " + str(b_43))
print ("b_32: " + str(b_32))
print ("b_21: " + str(b_21))

Theta: [[ 0.06253023]
 [-0.01487972]
 [-0.28486964]]
Theta_43: [[-9.94255529e-02  4.23195755e-01 -3.02006492e-01]
 [ 1.61060068e-01  5.86730450e-02  3.32050566e-03]
 [-2.01882237e-02 -1.70899744e-01  3.66044352e-01]
 [-1.54737915e-01 -3.92730503e-01 -1.02534180e-01]
 [-2.69227673e-01 -2.43616792e-05 -2.23089907e-01]]
Theta_32: [[ 0.04354973  0.28636852  0.05405697 -0.03337999  0.21674713]
 [ 0.24610642 -0.26511892 -0.14940207 -0.0758484   0.29655569]
 [ 0.21304746  0.01641003  0.23495516 -0.05855409  0.14937039]
 [ 0.13637395  0.06217949  0.01030109  0.00979549 -0.23933665]
 [ 0.0760769  -0.11660622  0.05139357 -0.31070488  0.2403705 ]
 [-0.11264108 -0.06255997 -0.14635127 -0.14745946  0.29195211]
 [ 0.13886869 -0.14666988  0.21746681  0.11431326  0.11738221]
 [-0.08852564 -0.07800745 -0.0463812   0.30717341 -0.04243015]]
Theta_21: [[ 0.13286708  0.00117581 -0.07153182 -0.11504868  0.40984214  0.03924191
   0.24171774  0.04667672]
 [-0.45095442  0.1400039   0.23078029  0.06121901 -0.08

The model can be pre-trained in the free version to narrow down to the shortest description length, but it doesn't eliminate the failed generations (examples don't observe the well-formed rules). The goal is to form the cache (narrow down the set further) to cater to the structure of the brain (Helmholtz machine) so that the generation is accurate (well within the phenotype boundary) while keeping the diversity of generated examples (avoid the dark-room problem).

There is a discrepency between the dataset and the machine capacity. The machine cannot process and generate the data distribution precisely. In order to fill in the gap, we either change the brain structure (the inner structure of the machine) or the data distribution. The sampling in active inference fulfills the later option. Here we assume the neuron connection of the brain is fixed (which is not true), namely the neural network is of some shitty, poor design (maybe some coelenterate's brain which is in a primitive stage and not been fully developed, which is a shittier analogy). But the parameters of the neuron connections are modifiable, so the internal representation is trainable which corresponds to the inference and model-fitting part of traditional statistical learning.

The brain is designed as a poor processor as a driven force to change the data. If the processor is too powerful, universal let's say, then any data distribution can be learned and recovered, there is no need to reshape the data distribution by active sampling because any input signal can be fully understand (means it can be represented internally and precisely recovered by synthesis). 

The stochastic sampling of binary neurons is helpful to construct the shortest description length and generate the data set in a distributional sense. It means that the distribution of the generated data should be asymptotically approaching the real distribution (ideally). However, down to each single sample, the accuracy of generation is not guaranteed. Unlike VAE which is governed by local variables, the Helmholtz machine is subject to global hierarchical controls which attributs to "1". It may not be a good idea to compact global and local variables into one control signal but it's the potential we want to explore in Helmholtz machine.

Question: how to design the training mechanism as fine-tuning? 
First, we have preferred set, which is much smaller than the well-formed set. The preferred set could be seen as a start point for the active sampling process, which means the preferred set doesn't conform to a distribution that the brain can simulate, thus the agent will active shaping the niche to conform to its cognition. Using preferred set as the initial niche, 

#### Generative Model (Preserved)

We first study the generative part (sleep phase) of Helmholtz machine. It works by pure generation with feedback (the discriminator designed by well-formed rules). It works like GAN but with generative model trained with deterministic feedback. Let's see how it works.

In [ ]:
def generation(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d,batch_size):
    p_4 = sigmoid(Theta)
    z = ((p_4 > np.random.rand(n_z,batch_size)).astype(int) - 0.5)*2    # rejection sampling
    
    p_3 = sigmoid(np.matmul(Theta_43,z) + b_43)
    y = ((p_3 > np.random.rand(n_y,batch_size)).astype(int) - 0.5)*2
    
    p_2 = sigmoid(np.matmul(Theta_32,y) + b_32)
    x = ((p_2 > np.random.rand(n_x,batch_size)).astype(int) - 0.5)*2
    # x = np.where(p_2 > 0.5,1,-1)
    
    p_1 = sigmoid(np.matmul(Theta_21,x) + b_21)
    # d = ((p_1 > np.random.rand(n_d,batch_size)).astype(int) - 0.5)*2
    d = np.where(p_1 > 0.5,1,-1)
    
    P_4 = np.cumprod(np.where(z == 1,p_4,1),axis=0)[-1] * np.cumprod(np.where(z == -1,1-p_4,1),axis=0)[-1]*(1.5**n_z)
    P_3 = np.cumprod(np.where(y == 1,p_3,1),axis=0)[-1] * np.cumprod(np.where(y == -1,1-p_3,1),axis=0)[-1]*(1.5**n_y)
    P_2 = np.cumprod(np.where(x == 1,p_2,1),axis=0)[-1] * np.cumprod(np.where(x == -1,1-p_2,1),axis=0)[-1]*(1.5**n_x)
    P_1 = np.cumprod(np.where(d == 1,p_1,1),axis=0)[-1] * np.cumprod(np.where(d == -1,1-p_1,1),axis=0)[-1]*(1.5**n_d)
    
    P = P_1 * P_2 * P_3 * P_4
    
    return p_4,p_3,p_2,p_1,z,y,x,d,P

In [ ]:
p_4_gen,p_3_gen,p_2_gen,p_1_gen,z_s_gen,y_s_gen,x_s_gen,d_s_gen,P_gen = generation(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,n_z,n_y,n_x,n_d,test_size)
print ("p_4_gen: " + str(p_4_gen[:,0:10]))
print ("p_3_gen: " + str(p_3_gen[:,0:10]))
print ("p_2_gen: " + str(p_2_gen[:,0:10]))
print ("p_1_gen: " + str(p_1_gen[:,0:10]))
print ("z_s_gen: " + str(z_s_gen[:,0:10]))
print ("y_s_gen: " + str(y_s_gen[:,0:10]))
print ("x_s_gen: " + str(x_s_gen[:,0:10]))
print ("d_s_gen: " + str(d_s_gen[:,0:10]))
print ("P_gen: " + str(P_gen[0:10]))

In [633]:
 # check if "d_s_gen" is in the well-formed set
def check(d_s_gen):
    d_s = 0
    pattern_len = d_s_gen.shape[0]
    for j in range(d_s_gen.shape[1]):
        flag = 0
        pattern = d_s_gen[:,j]
        if pattern[0] != 1:
            flag = 1
            continue
        if np.array_equal(pattern[0:3], [1,-1,-1]) or np.array_equal(pattern[-3:], [-1,-1,1]):
            flag = 1
            continue
        for k in range(pattern_len - 5):
            if np.array_equal(pattern[k:k+5],[-1,-1,1,-1,-1]):
                flag = 1
                break
        if flag == 1:
            continue
        for k in range(pattern_len - 4):
            if np.array_equal(pattern[k:k+4],[-1,-1,-1,-1]):
                flag = 1
                break

        if flag == 0:
            d_s = d_s_gen[:,j:j+1]
            break
    return d_s,j,flag

In [ ]:
def gen_update_delta(z,y,x,d,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr):
    p_4 = sigmoid(Theta)
    Theta -= lr * (p_4 - (1+z)/2)
    
    p_3 = sigmoid(np.matmul(Theta_43,z) + b_43)
    Theta_43 -= lr * np.outer((p_3 - (1+y)/2), z)
    b_43 -= lr * (p_3 - (1+y)/2)
    
    p_2 = sigmoid(np.matmul(Theta_32,y) + b_32)
    Theta_32 -= lr * np.outer((p_2 - (1+x)/2), y)
    b_32 -= lr * (p_2 - (1+x)/2)
    
    p_1 = sigmoid(np.matmul(Theta_21,x) + b_21)
    Theta_21 -= lr * np.outer((p_1 - (1+d)/2), x)
    b_21 -= lr * (p_1 - (1+d)/2)
    
    return Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21